In [ ]:
# !pip install torch==1.10.1+cu111 torchvision==0.11.2+cu111 torchaudio==0.10.1 -f https://download.pytorch.org/whl/torch_stable.html
!pip install timm # install pytorch image models
!pip install torchmetrics

In [ ]:
import torch
import os
import pandas as pd
import numpy as np
import random 

import albumentations as A #图像增强库
import cv2
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn import preprocessing
from sklearn.model_selection import StratifiedKFold
import timm

import torchvision
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torchvision.models as models
import torch.nn.functional as F
from torch import nn
import torchmetrics 
from torch.nn.modules.loss import _Loss

In [ ]:
!nvidia-smi

In [ ]:
class GlobalConstantsConfigure():
    def __init__(self):
        self.continue_training = False
        self.last_model = '../input/sorghum-cultivar-identification-models/eff_v2_sgd_50_silu.pt' 
        self.num_epochs_done = 0
        self.seed = 107
        self.fold = 1
        self.num_folds = 4
        self.num_classes = 100 #一百分类问题
        self.biggest_loss = 999
        self.training_size_rate = 0.8
        self.training_dir = '../input/sorghum-id-fgvc-9/train_images'
        self.model_name = 'tf_efficientnetv2_m_in21k'#模型名称
        self.model_path = './efficientnetv2_b5_sgd_50.pt'
        self.image_size = 512
        self.batch_size = 8
        self.val_batch_size = 32
        self.lr = 3e-5 # 3e-5
        self.num_epochs = 25
        self.steps_per_decay = 5
        self.device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
        self.num_workers = 2  # if torch.cuda.is_available() else 4
gcc = GlobalConstantsConfigure()

In [ ]:
def set_seed(seed) : #设置随机数种子，方便训练周期的复现，确保每一个epoch的卷积的输入输出都是一致的
    os.environ['PYTHONHASHSEED'] = str(seed) #禁止hash随机化，使得实验可复现
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)#CPU设置生成固定随机数的种子
    torch.cuda.manual_seed(seed)#GPU设置固定生成随机数的种子
    torch.backends.cudnn.deterministic = True#表示返回的卷积算法是确定的，配合上面的随机数种子的设置，可以确定每次运行网络的时候相同输入的输出是固定的
    torch.backends.cudnn.benchmark = True #让内置的 cuDNN 的 auto-tuner 自动寻找最适合当前配置的高效算法，来达到优化运行效率的问题。
set_seed(gcc.seed)

In [ ]:
df_all = pd.read_csv('../input/sorghum-id-fgvc-9/train_cultivar_mapping.csv')#数据读取
print(len(df_all))
df_all.dropna(inplace=True)#，数据预处理：删除数据中空值的数据
print(len(df_all))
df_all.head()

In [ ]:
unique_cultivars = list(df_all["cultivar"].unique())

In [ ]:
#数据加载
df_all["file_path"] = df_all["image"].apply(lambda image: '../input/sorghum-id-fgvc-9/train_images/' + image)
df_all["cultivar_index"] = df_all["cultivar"].map(lambda item: unique_cultivars.index(item))#根据标签从小到大排序
df_all["is_exist"] = df_all["file_path"].apply(lambda file_path: os.path.exists(file_path))
df_all = df_all[df_all.is_exist==True]#拿到所有file_path为True字段的数据
df_all.head()

In [ ]:
skf = StratifiedKFold(n_splits=gcc.num_folds, shuffle=True, random_state=gcc.seed)
#分层随机抽样，验证集中不同类别占比与原始样本的比例保持一致


train_folds = []
val_folds = []

for train_idx, valid_idx in skf.split(df_all['image'], df_all["cultivar_index"]):#训练集与验证集的划分
    train_folds.append(train_idx)
    val_folds.append(valid_idx)
#     df_train = df_all.iloc[train_idx]
#     df_valid = df_all.iloc[valid_idx]

# print(train_folds)
# print(val_folds)
df_train = df_all.iloc[train_folds[gcc.fold]]
df_valid = df_all.iloc[val_folds[gcc.fold]]




print(f"train size: {len(df_train)}")#train size: 16645
print(f"valid size: {len(df_valid)}")#valid size: 5548

print(df_train.cultivar.value_counts())#打印训练集各类标签的总数
print(df_valid.cultivar.value_counts())

In [ ]:
class SorghumDataset(Dataset):#高粱数据类
    def __init__(self, dirs, labels, transformation=None):
        super(SorghumDataset,self).__init__()
        self.dirs = dirs
        self.labels = labels
        self.transformation = transformation
    def __len__(self):
        return len(self.dirs)

    def __getitem__(self, index):
        image = cv2.imread(self.dirs[index])
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)#图像二值化，方便后续的预处理操作
        label = self.labels[index] # need to one hot encoding here独热编码
        
        image = np.array(image)

        if self.transformation:
            aug_image = self.transformation(image=image) #做一些图像增强的操作
            image = aug_image['image']
            
        image = image / 255. #归一化
        image = image.transpose((2, 0, 1))#索引置换，矩阵置换
        
        image = torch.from_numpy(image).type(torch.float32)#把数组转换成张量tensor，且二者共享内存，对张量进行修改比如重新赋值，那么原始数组也会相应发生改变。
        image = transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))(image) #将这些范围内的数据，转化为-1~1之间
        
        labels = torch.from_numpy(np.array(self.labels[index])).type(torch.float32)#label数组转为张量tensor


        return image, labels

In [ ]:
training_transformation = A.Compose([ #做图像增强
    A.Resize(width=gcc.image_size, height=gcc.image_size, p=1.0),#图像重定义大小size
    A.Flip(p=0.5),
    A.RandomRotate90(p=0.5),#图像随机旋转90°
    A.ShiftScaleRotate(p=0.5),#改变旋转尺度
    A.HueSaturationValue(p=0.5),#调整图片色相饱和度
#     A.OneOf([
#         A.RandomBrightnessContrast(p=0.5),
#         A.RandomGamma(p=0.5),
#     ], p=0.5),
#     A.OneOf([
#         A.Blur(p=0.1),
#         A.GaussianBlur(p=0.1),
#         A.MotionBlur(p=0.1),
#     ], p=0.1),
#     A.OneOf([
#         A.GaussNoise(p=0.1),
#         A.ISONoise(p=0.1),
#         A.GridDropout(ratio=0.5, p=0.2),
#         A.CoarseDropout(max_holes=16, min_holes=8, max_height=16, max_width=16, min_height=8, min_width=8, p=0.2)
#     ], p=0.2),

])
validation_transformation = A.Compose([
    A.Resize(width=gcc.image_size, height=gcc.image_size, p=1.0)
])

In [ ]:
# training_set = SorghumDataset(training_dirs, training_labels, training_transformation)
# validation_set = SorghumDataset(validation_dirs, validation_labels, validation_transformation)

#数据划分并初始化

training_set = SorghumDataset(df_train.file_path.values, df_train.cultivar_index.values, training_transformation)
validation_set = SorghumDataset(df_valid.file_path.values, df_valid.cultivar_index.values, validation_transformation)


training_dataloader = DataLoader(
    training_set,
    batch_size = gcc.batch_size,
    shuffle = True,
    num_workers = gcc.num_workers,
    pin_memory = True, 
    drop_last = True
)
validation_dataloader = DataLoader(
    validation_set,
    batch_size = gcc.val_batch_size,
    shuffle = True,
    num_workers = gcc.num_workers,
    pin_memory = True,
    drop_last = True
)

In [ ]:
class CustomModel(torch.nn.Module):#定义模型 
    def __init__(self, model_backbone):
        super(CustomModel,self).__init__()
        self.model = model_backbone#主干网络，用来做特征提取的网络，代表网络的一部分，一般用于前端提取图片信息，生成特征图feature map，供后面的网络使用。
        
        self.model.classifier = nn.Sequential(#网络定义
            nn.BatchNorm1d(1280),#特征归一化，取1280个特征
            nn.Linear(1280, 512),#定义全连接层，输出为512
            nn.Dropout(0.5),#dropout层，比例为0.5，防止过拟合
            nn.ReLU(inplace=True),#激活函数
            nn.Linear(512, gcc.num_classes),
            
#             nn.BatchNorm1d(1280),
#             nn.Linear(1280, 512),
#             nn.Dropout(0.5),
#             nn.SiLU(inplace=True),

#             nn.Linear(512, 256),
#             nn.Dropout(0.5),
#             nn.SiLU(inplace=True),
#             nn.Linear(256, gcc.num_classes)
        )
    def forward(self,x):
        x = self.model(x)
        return x

In [ ]:
def to_one_hot(labels, num_classes, dtype=torch.float, dim=1):
    if labels.ndim < dim + 1:
        shape = list(labels.shape) + [1] * (dim + 1 - len(labels.shape))
        labels = torch.reshape(labels, shape)
    sh = list(labels.shape)
    sh[dim] = num_classes
    o = torch.zeros(size=sh, dtype=dtype, device=labels.device)
    labels = o.scatter_(dim=dim, index=labels.long(), value=1)
    return labels


class PolyLoss(_Loss):
    def __init__(self, softmax, ce_weight=None, reduction='mean', epsilon=1.0):
        super().__init__()
        self.softmax = softmax
        self.reduction = reduction
        self.epsilon = epsilon
        self.cross_entropy = nn.CrossEntropyLoss(weight=ce_weight, reduction='none')

    def forward(self, input, target):

        if len(input.shape) - len(target.shape) == 1:
            target = target.unsqueeze(1).long()
        n_pred_ch, n_target_ch = input.shape[1], target.shape[1]
        if n_pred_ch != n_target_ch:
            self.ce_loss = self.cross_entropy(input, torch.squeeze(target, dim=1).long())
            target = to_one_hot(target, num_classes=n_pred_ch)
        else:
            self.ce_loss = self.cross_entropy(input, torch.argmax(target, dim=1))

        if self.softmax:
            input = torch.softmax(input, 1)

        pt = (input * target).sum(dim=1) 
        
        poly_loss = self.ce_loss + self.epsilon * (1 - pt)

        polyl = torch.mean(poly_loss)  # the batch and channel average
        # polyl = torch.sum(poly_loss)  # sum over the batch and channel dims
        return (polyl)

In [ ]:
# backbone = models.efficientnet_b5(pretrained=True) 
backbone = timm.create_model(gcc.model_name,pretrained=True)#主干网络


# print(index)
model = CustomModel(backbone)
ce = torch.nn.CrossEntropyLoss()
loss_func = PolyLoss(ce) # torch.nn.CrossEntropyLoss()
metrics_acc = torchmetrics.Accuracy(threshold=0.0, num_classes = gcc.num_classes)
print(model)


# for index, child in enumerate(backbone.children()):
#     print(index)
#     if index <= 7:
#         for param in child.parameters():
#             param.requires_grad = False


trainable_parameters = [param for param in model.parameters() if param.requires_grad == True]
optimizer = torch.optim.Adam(trainable_parameters, lr = gcc.lr)
# optimizer = torch.optim.SGD(trainable_parameters, lr = gcc.lr, momentum = 0.9)
# optimizer = torch.optim.SGD(trainable_parameters, lr = gcc.lr, momentum=0.9)
# lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, gcc.steps_per_decay)
lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=2)

model.to(gcc.device)

if gcc.continue_training == True:
    # model.load_state_dict(torch.load(gcc.last_model))
    checkpoint = torch.load(gcc.last_model)
    
    model.load_state_dict(checkpoint['model_state_dict'])
    
    
    # optimizer = torch.optim.Adam(trainable_parameters, lr = gcc.lr)
    # optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    
    # lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, gcc.steps_per_decay)
    # lr_scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
    
    # print(lr_scheduler.state_dict())

print('load model done')

In [ ]:
# def calc_accuracy(pred, true):
#     # print(pred, true)
#     true = true.type(torch.int64) # label
#     pred = F.softmax(pred, dim = 1)
#     true = torch.zeros(pred.shape[0], pred.shape[1]).scatter_(1, true.unsqueeze(1), 1.)
#     acc = (true.argmax(-1) == pred.argmax(-1)).float().detach().numpy()
#     acc = float(acc.sum() / len(acc))
#     return round(acc, 4)

In [ ]:
def training_progress(training_dataloader, loss_func, scheduler):
    model.train()
    training_loss = 0
    training_acc = 0
    cnt = 0 
    print('Learning rate: ',scheduler.get_last_lr())
    print(scheduler.state_dict())
    training_loader = tqdm(training_dataloader, desc='Iterating through the training set')
    for image, label in training_loader:
        image = image.to(gcc.device)
        label = label.to(gcc.device)
        
        output = model(image)
        # output.to(gcc.device)

        acc = metrics_acc(output.cpu().argmax(1), label.cpu().int())
        loss = loss_func(output, label.long())
        # calculate accuracy here

        training_loss += loss.item()
        training_acc += acc
        cnt +=1 
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        scheduler.step()
    
    mean_training_loss = training_loss / cnt
    mean_training_acc = training_acc / cnt
    
    return mean_training_loss, mean_training_acc

In [ ]:
def validation_progress(validation_dataloader, loss_func):
    model.eval()
    validation_loss = 0
    validation_acc = 0
    cnt = 0 
    validation_loader = tqdm(validation_dataloader, desc='Iterating through the validation set')
    with torch.no_grad():
        for image, label in validation_loader:
            image = image.to(gcc.device)
            label = label.to(gcc.device)

            output = model(image)
            loss = loss_func(output, label.long())
            # acc = calc_accuracy(output.cpu(), label.cpu())
            # output.to(gcc.device)
            acc = metrics_acc(output.cpu().argmax(1), label.cpu().int())
            # calculate accuracy here
            validation_loss += loss.item()
            validation_acc += acc
            
            cnt += 1

    mean_validation_loss = validation_loss / cnt
    mean_validation_acc = validation_acc / cnt
    return mean_validation_loss, mean_validation_acc

In [ ]:
def training_model(model, training_dataloader, validation_dataloader, loss_func, scheduler):
    training_losses_history, validation_losses_history = [], []
    training_acc_history, validation_acc_history = [], []
    best_loss = gcc.biggest_loss
    for epoch in range(gcc.num_epochs):
        
        training_loss, training_acc = training_progress(training_dataloader, loss_func, scheduler)
        training_losses_history.append(training_loss)
        training_acc_history.append(training_acc)
        
        validation_loss, validation_acc = validation_progress(validation_dataloader, loss_func)
        validation_losses_history.append(validation_loss)
        validation_acc_history.append(validation_acc)
        
        if validation_loss <= best_loss: # sussy baka
            best_loss = validation_loss
            torch.save({
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'scheduler_state_dict': scheduler.state_dict()
            }, gcc.model_name + '_best.pt')
            # torch.save(model.state_dict(), gcc.model_name + '_best.pt')
        
        if epoch == gcc.num_epochs - 1: # i believe my timing capability
            torch.save({
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'scheduler_state_dict': scheduler.state_dict()
            }, gcc.model_name + '_' + str(gcc.num_epochs_done + gcc.num_epochs) + '_last.pt')

        print(f'Epoch {epoch + 1}/{gcc.num_epochs} | Training_loss : {training_loss:.3f} | Validation_loss : {validation_loss:.3f}' 
             + f' Training_acc : {training_acc:.3f} | Validation_acc : {validation_acc:.3f}'
             )
    return training_losses_history, validation_losses_history, training_acc_history, validation_acc_history

In [ ]:
training_losses_history, validation_losses_history, training_acc_history, validation_acc_history = training_model(model, training_dataloader, validation_dataloader, loss_func, lr_scheduler)

In [ ]:
# testing_progress(testing_dataloader, loss_func)

In [ ]:
def plot_loss_history(model_name, train_loss_history, val_loss_history, num_epochs):
    
    x = np.arange(num_epochs)
    fig = plt.figure(figsize=(10, 6))
    plt.plot(x, train_loss_history, label='Train Loss', lw=3)
    plt.plot(x, val_loss_history, label='Validation Loss', lw=3)

    plt.title(f"{model_name}", fontsize=20)
    plt.legend(fontsize=12)
    plt.xlabel("Epoch", fontsize=15)
    plt.ylabel("Loss", fontsize=15)

    plt.show()
    
plot_loss_history(gcc.model_name, training_losses_history, validation_losses_history, gcc.num_epochs)

In [ ]:
def plot_acc_history(model_name, train_acc_history, val_acc_history, num_epochs):
    
    x = np.arange(num_epochs)
    fig = plt.figure(figsize=(10, 6))
    plt.plot(x, train_acc_history, label='Training Accuracy', lw=3)
    plt.plot(x, val_acc_history, label='Validation Accuracy', lw=3)

    plt.title(f"{model_name}", fontsize=20)
    plt.legend(fontsize=12)
    plt.xlabel("Epoch", fontsize=15)
    plt.ylabel("Accuracy", fontsize=15)

    plt.show()
    
plot_acc_history(gcc.model_name, training_acc_history, validation_acc_history, gcc.num_epochs)

In [ ]:
checkpoint = torch.load(gcc.model_name + '_best.pt')
model.load_state_dict(checkpoint['model_state_dict'])

In [ ]:
sub = pd.read_csv('../input/sorghum-id-fgvc-9/sample_submission.csv')
sub.head()

In [ ]:
sub["filename"] = sub["filename"].apply(lambda image: '../input/sorghum-id-fgvc-9/test/' + image)
sub["cultivar"] = 0
sub.head()

In [ ]:
testing_dataset = SorghumDataset(sub['filename'], sub['cultivar'], validation_transformation)
testing_dataloader = DataLoader(testing_dataset, 
                                batch_size=gcc.val_batch_size, 
                                shuffle=False, 
                                num_workers=gcc.num_workers)

In [ ]:
# predictions = np.zeros(len(testing_dataloader))
predictions = []
cnt = 0 
with torch.no_grad():
    for image, label in tqdm(testing_dataloader):
        image = image.to(gcc.device)
        outputs = model(image)
        # print(outputs)
        preds = outputs.detach().cpu()
        predictions.append(preds.argmax(1)) # need optimize here
        # print(predictions)

In [ ]:
tmp = predictions[0]
for i in range(len(predictions) - 1):
    tmp = torch.cat((tmp, predictions[i+1]))

In [ ]:
# predictions = label_encoder.inverse_transform(tmp)
predictions = [unique_cultivars[pred] for pred in tmp]

In [ ]:
sub = pd.read_csv('../input/sorghum-id-fgvc-9/sample_submission.csv')
sub['cultivar'] = predictions
sub.to_csv('../output/kaggle/working/submission.csv', index=False)
sub.head()